In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import cv2 as cv
import numpy as np

from typing import List, Tuple
from skimage.measure import approximate_polygon
from sympy import Segment, Point, Line, N, acos

In [3]:
img = cv.imread('test_picture_3.JPG')
img_gray = cv.imread('test_picture_3.JPG', 0)

ret, thresh = cv.threshold(img_gray, 230, 200, 0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
arrow_contour = np.squeeze(contours[6])
approx_contour = approximate_polygon(arrow_contour, tolerance=2.5)

In [12]:
contour = approx_contour 
contour

array([[648,  79],
       [819,  78],
       [826,  81],
       [835,  93],
       [835, 230],
       [830, 239],
       [819, 245],
       [649, 245],
       [640, 240],
       [634, 230],
       [634,  93],
       [647,  79]], dtype=int32)

In [13]:
segments = []
lengths = []
points = []
for i in range(len(contour)):
    point1 = contour[i]
    point2 = contour[i + 1] if i < len(contour) - 1 else contour[0]
    p1 = Point(point1[0], point1[1])
    p2 = Point(point2[0], point2[1])
    segment = Segment(p1, p2)
    if p1 not in points:
        points.append(p1)
    if p2 not in points:
        points.append(p2)
    segments.append(segment)
    lengths.append(N(segment.length))

In [9]:
segments

[Segment2D(Point2D(648, 79), Point2D(819, 78)),
 Segment2D(Point2D(819, 78), Point2D(826, 81)),
 Segment2D(Point2D(826, 81), Point2D(835, 93)),
 Segment2D(Point2D(835, 93), Point2D(835, 230)),
 Segment2D(Point2D(835, 230), Point2D(830, 239)),
 Segment2D(Point2D(830, 239), Point2D(819, 245)),
 Segment2D(Point2D(819, 245), Point2D(649, 245)),
 Segment2D(Point2D(649, 245), Point2D(640, 240)),
 Segment2D(Point2D(640, 240), Point2D(634, 230)),
 Segment2D(Point2D(634, 230), Point2D(634, 93)),
 Segment2D(Point2D(634, 93), Point2D(647, 79)),
 Segment2D(Point2D(647, 79), Point2D(648, 79))]

In [10]:
lengths

[171.002923951610,
 7.61577310586391,
 15.0000000000000,
 137.000000000000,
 10.2956301409870,
 12.5299640861417,
 170.000000000000,
 10.2956301409870,
 11.6619037896906,
 137.000000000000,
 19.1049731745428,
 1.00000000000000]

In [14]:
points

[Point2D(648, 79),
 Point2D(819, 78),
 Point2D(826, 81),
 Point2D(835, 93),
 Point2D(835, 230),
 Point2D(830, 239),
 Point2D(819, 245),
 Point2D(649, 245),
 Point2D(640, 240),
 Point2D(634, 230),
 Point2D(634, 93),
 Point2D(647, 79)]

In [15]:
lengths = sorted(lengths)
threshold = 0
for i in range(len(lengths) - 1):
    length = lengths[i]
    if length < 3:
        continue
    next_length = lengths[i + 1]
    if next_length / length > 4:
        threshold = next_length
        break

In [27]:
lengths[8]/lengths[7]

7.17090773948593

In [16]:
threshold

137.000000000000

In [28]:
short_segments = [segment for segment in segments if N(segment.length) < threshold]
singles = list(short_segments)

In [29]:
short_segments

[Segment2D(Point2D(819, 78), Point2D(826, 81)),
 Segment2D(Point2D(826, 81), Point2D(835, 93)),
 Segment2D(Point2D(835, 230), Point2D(830, 239)),
 Segment2D(Point2D(830, 239), Point2D(819, 245)),
 Segment2D(Point2D(649, 245), Point2D(640, 240)),
 Segment2D(Point2D(640, 240), Point2D(634, 230)),
 Segment2D(Point2D(634, 93), Point2D(647, 79)),
 Segment2D(Point2D(647, 79), Point2D(648, 79))]

In [30]:
singles

[Segment2D(Point2D(819, 78), Point2D(826, 81)),
 Segment2D(Point2D(826, 81), Point2D(835, 93)),
 Segment2D(Point2D(835, 230), Point2D(830, 239)),
 Segment2D(Point2D(830, 239), Point2D(819, 245)),
 Segment2D(Point2D(649, 245), Point2D(640, 240)),
 Segment2D(Point2D(640, 240), Point2D(634, 230)),
 Segment2D(Point2D(634, 93), Point2D(647, 79)),
 Segment2D(Point2D(647, 79), Point2D(648, 79))]

In [32]:
type(singles)

list

In [33]:
type(short_segments)

list

In [34]:
len(singles)

8

In [36]:
def segments_have_common_point(s1: Segment, s2: Segment) -> bool:
    return s1.p1 == s2.p1 or s1.p1 == s2.p2 or s1.p2 == s2.p1 or s1.p2 == s2.p2

In [37]:
def segments_in_group(s1: Segment, s2: Segment, group: List[Segment]) -> Segment:
    segments = [s1, s2]
    if s1 in group:
        segments.remove(s1)
    if s2 in group:
        segments.remove(s2)
    if len(segments) < 2:
        return segments
    return None

In [38]:
adjacent_segments_groups = []

for i in range(len(short_segments)):
    for j in range(len(short_segments)):
        if i == j:
            continue
        s1 = short_segments[i]
        s2 = short_segments[j]
        if segments_have_common_point(s1, s2):
            if s1 in singles:
                singles.remove(s1)
            if s2 in singles:
                singles.remove(s2)
            found = False
            for group in adjacent_segments_groups:
                segment_to_add = segments_in_group(s1, s2, group)
                if segment_to_add is not None:
                    group.extend(segment_to_add)
                    found = True
            if not found:
                adjacent_segments_groups.append([s1, s2])

In [39]:
adjacent_segments_groups

[[Segment2D(Point2D(819, 78), Point2D(826, 81)),
  Segment2D(Point2D(826, 81), Point2D(835, 93))],
 [Segment2D(Point2D(835, 230), Point2D(830, 239)),
  Segment2D(Point2D(830, 239), Point2D(819, 245))],
 [Segment2D(Point2D(649, 245), Point2D(640, 240)),
  Segment2D(Point2D(640, 240), Point2D(634, 230))],
 [Segment2D(Point2D(634, 93), Point2D(647, 79)),
  Segment2D(Point2D(647, 79), Point2D(648, 79))]]

In [43]:
adjacent_segments_groups[1][1].midpoint

Point2D(1649/2, 242)

In [44]:
def get_segment_midpoint(segment: Segment) -> Point:
    x = (segment.p1[0] + segment.p2[0]) // 2
    y = (segment.p1[1] + segment.p2[1]) // 2
    return Point(x, y)

In [45]:
get_segment_midpoint(adjacent_segments_groups[1][1])

Point2D(824, 242)

In [4]:
mean_list = [1,2,3,4,5]
np.mean(mean_list)

3.0

In [5]:
def mean(lst: list) -> float:
    return sum(lst) / len(lst)

In [6]:
mean(mean_list)

3.0

In [4]:
def segments_have_common_point(s1: Segment, s2: Segment) -> bool:
    return s1.p1 == s2.p1 or s1.p1 == s2.p2 or s1.p2 == s2.p1 or s1.p2 == s2.p2

In [5]:
def get_segment_midpoint(segment: Segment) -> Point:
    x = (segment.p1[0] + segment.p2[0]) // 2
    y = (segment.p1[1] + segment.p2[1]) // 2
    return Point(x, y)

In [6]:
def segments_in_group(s1: Segment, s2: Segment, group: List[Segment]) -> Segment:
    segments = [s1, s2]
    if s1 in group:
        segments.remove(s1)
    if s2 in group:
        segments.remove(s2)
    if len(segments) < 2:
        return segments
    return None

In [7]:
def collect_adjacent_segment_groups(segments: List[Segment], threshold: float) -> List[Tuple[Segment, Segment]]:
    adjacent_segments_groups = []
    short_segments = [segment for segment in segments if N(segment.length) < threshold]
    singles = list(short_segments)
    for i in range(len(short_segments)):
        for j in range(len(short_segments)):
            if i == j:
                continue
            s1 = short_segments[i]
            s2 = short_segments[j]
            if segments_have_common_point(s1, s2):
                if s1 in singles:
                    singles.remove(s1)
                if s2 in singles:
                    singles.remove(s2)
                found = False
                for group in adjacent_segments_groups:
                    segment_to_add = segments_in_group(s1, s2, group)
                    if segment_to_add is not None:
                        group.extend(segment_to_add)
                        found = True
                if not found:
                    adjacent_segments_groups.append([s1, s2])
    return adjacent_segments_groups

In [8]:
def collect_segments_lengths(contour):
    segments = []
    lengths = []
    points = []
    for i in range(len(contour)):
        point1 = contour[i]
        point2 = contour[i + 1] if i < len(contour) - 1 else contour[0]
        p1 = Point(point1[0], point1[1])
        p2 = Point(point2[0], point2[1])
        segment = Segment(p1, p2)
        if p1 not in points:
            points.append(p1)
        if p2 not in points:
            points.append(p2)
        segments.append(segment)
        lengths.append(N(segment.length))
    return segments, points, lengths

In [9]:
def find_splitting_threshold(lengths: List[float]) -> float:
    lengths = sorted(lengths)
    threshold = 0
    for i in range(len(lengths) - 1):
        length = lengths[i]
        if length < 3:
            continue
        next_length = lengths[i + 1]
        if next_length / length > 4:
            threshold = next_length
            break
    return threshold

In [10]:
def compute_roi_center(group: List[Segment]) -> List[Point]:
    buffer = list(group)
    while True:
        if len(buffer) == 1:
            midpoint = get_segment_midpoint(buffer[0])
            return midpoint
        
        points = []
        for segment in buffer:
            midpoint = get_segment_midpoint(segment)
            points.append(midpoint)

        buffer = []
        while len(points) > 1:
            point1 = points.pop(0)
            point2 = points.pop(0)
            buffer.append(Segment(point1, point2))

        if len(points) == 1:
            buffer.append(Segment(point2, points[0]))

In [11]:
def find_roi(contour):
    roi_centers = []
    segments, points, lengths = collect_segments_lengths(contour)

    threshold = find_splitting_threshold(lengths)

    if threshold > 0:
        adjacent_segments_groups = collect_adjacent_segment_groups(segments, threshold)

        for group in adjacent_segments_groups:
            for segment in group:
                if segment.p1 in points:
                    del points[points.index(segment.p1)]
                if segment.p2 in points:
                    del points[points.index(segment.p2)]

        for group in adjacent_segments_groups:
            roi_center = compute_roi_center(group)
            roi_centers.append(roi_center)

    roi_centers.extend(points)

    return roi_centers

In [12]:
def main():
    img = cv.imread('test_picture_3.JPG')
    img_gray = cv.imread('test_picture_3.JPG', 0)
    
    ret, thresh = cv.threshold(img_gray, 230, 200, 0)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    arrow_contour = np.squeeze(contours[6])
    approx_contour = approximate_polygon(arrow_contour, tolerance=2.5)
    roi = find_roi(approx_contour)

    for point in roi:
        cv.circle(img, (int(point[0]), int(point[1])), 4, (0, 0, 255), -1)

    for point in approx_contour:
        cv.circle(img, (point[0], point[1]), 2, (0, 0, 0), -1)

    cv.imwrite('contoured.jpg', img)
    cv.imshow('Image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [13]:
if __name__ == '__main__':
    main()